# Certamen 2 - Computacion Cientifica

## Pregunta 1

In [1]:
import numpy as np
import scipy.interpolate

In [2]:
N = 5
M = 3

Construccion de Factorizacion de array $T_{n}$

In [3]:
def create_U(n: int) -> np.ndarray:    
    UB = np.zeros((n, n))
    for i in np.arange(n):
        UB[i,i] = (i+2)/(i+1)
        for j in np.arange(n):
            if i-j == -1:
                UB[i,j] = -1
    return UB

def create_L(n: int) -> np.ndarray:
    LB = np.zeros((n,n))
    for i in np.arange(n):
        LB[i,i] = 1
        for j in np.arange(n):
            if i-j == 1:
                LB[i,j] = -(i/(i+1))
    return LB

In [4]:
L = create_L(N)
U = create_U(N)

In [5]:
print(L)
print(U)

[[ 1.          0.          0.          0.          0.        ]
 [-0.5         1.          0.          0.          0.        ]
 [ 0.         -0.66666667  1.          0.          0.        ]
 [ 0.          0.         -0.75        1.          0.        ]
 [ 0.          0.          0.         -0.8         1.        ]]
[[ 2.         -1.          0.          0.          0.        ]
 [ 0.          1.5        -1.          0.          0.        ]
 [ 0.          0.          1.33333333 -1.          0.        ]
 [ 0.          0.          0.          1.25       -1.        ]
 [ 0.          0.          0.          0.          1.2       ]]


In [6]:
T = L @ U
print(T)

[[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]


### Funciones para generar puntos equiespaciados y chevishev

In [7]:
def equiespaciados(n):
    equis = ((2 * (np.arange(1, n+1) - 1)) / (n-1)) - 1
    return equis

def chev(n):
    upper = ((2*np.arange(1, n+1)) - 1) * np.pi
    lower = 2 * n
    chev = np.cos(upper/lower)
    return chev

### Funciones para aplicar forward substitution y backward substitution

In [8]:
def backward_substitution(A: np.ndarray, b: np.ndarray, n: int):
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        for j in range(i, n):
            b[i] = b[i] - A[i,j]*x[j]
        x[i] = b[i]/A[i,i]
    return x

def forward_substitution(A: np.ndarray, b: np.ndarray, n: int):
    x = np.zeros(n)
    x[0] = (1./A[0,0]) * b[0]
    for i in range(1,n):
        x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x

### Funcion a aproximar, usada con propositos de prueba

In [9]:
f = lambda x: np.cos(2*x)

### Creacion de $n$ puntos equiespaciados y su evalucion en f() para pruebas

In [10]:
x_known = equiespaciados(N)
y_unknown = f(x_known)
print(x_known)
print(y_unknown)

[-1.  -0.5  0.   0.5  1. ]
[-0.41614684  0.54030231  1.          0.54030231 -0.41614684]


### Creacion de puntos conocidos

In [11]:
x_lin = np.linspace(-1, 1, M)
y_lin = f(x_lin)
print(x_lin)
print(y_lin)

[-1.  0.  1.]
[-0.41614684  1.         -0.41614684]


In [12]:
x_chev = chev(M)
y_chev = f(x_chev)
print(x_chev)
print(y_chev)


[ 8.66025404e-01  6.12323400e-17 -8.66025404e-01]
[-0.16055654  1.         -0.16055654]


### Estimacion de $w$ con los dos set de puntos

In [13]:
y_lin_app = scipy.interpolate.barycentric_interpolate(x_lin, y_lin, x_known)
print(f"Original      = {y_unknown}")
print(f"Interpolacion = {y_lin_app}")
print(f"Error         = {y_unknown - y_lin_app}")
print(f"Error Total   = {np.sum(np.abs(y_unknown - y_lin_app))}")

Original      = [-0.41614684  0.54030231  1.          0.54030231 -0.41614684]
Interpolacion = [-0.41614684  0.64596329  1.          0.64596329 -0.41614684]
Error         = [ 0.         -0.10566098  0.         -0.10566098  0.        ]
Error Total   = 0.2113219699901493


In [14]:
y_chev_app = scipy.interpolate.barycentric_interpolate(x_chev, y_chev, x_known)
print(f"Original      = {y_unknown}")
print(f"Interpolacion = {y_chev_app}")
print(f"Error         = {y_unknown - y_chev_app}")
print(f"Error Total   = {np.sum(np.abs(y_unknown - y_chev_app))}")

Original      = [-0.41614684  0.54030231  1.          0.54030231 -0.41614684]
Interpolacion = [-0.54740872  0.61314782  1.          0.61314782 -0.54740872]
Error         = [ 0.13126188 -0.07284551  0.         -0.07284551  0.13126188]
Error Total   = 0.40821479231881685


In [15]:
if np.sum(np.abs(y_unknown - y_chev_app)) < np.sum(np.abs(y_unknown - y_lin_app)):
    pri = "Chevishev"
else:
    pri = "Equispaciado"

print(f"Best: {pri}")

Best: Equispaciado


### Aproximacion de $v$

In [16]:
c_chev = forward_substitution(L, y_chev_app, N)
v_chev = backward_substitution(U, c_chev, N)
print(v_chev)

[0.5657391  1.67888692 2.17888692 1.67888692 0.5657391 ]


In [17]:
c_original = forward_substitution(L, y_unknown, N)
v_original = backward_substitution(U, c_original, N)
print(v_original)

[0.62415547 1.66445778 2.16445778 1.66445778 0.62415547]


In [18]:
print(f"Error Total   = {np.sum(np.abs(v_original - v_chev))}")

Error Total   = 0.16012017687540836


## Pregunta 2

### Create test data

In [19]:
x = np.array([0.1, 0.8, 0.7, 1.5, 2.3, 2.5, 2.7, 3.1, 3.8, 4.4])
y = np.array([0.5, 1., 0.2, 1.3, 1.2, 2.2, 1.6, 2, 2.3, 3.7])
t = np.array([0.08, 0.3, 0.35, 0.75, 1.2, 1.25, 1.3, 1.55, 1.9, 2.5])

### Point to work with

In [20]:
p = np.array([2.5 , 3]) 

### Create matriz a to solve square means problems

In [21]:
A = np.vstack([np.ones(x.size), t])
A = A.T

In [22]:
print(A)

[[1.   0.08]
 [1.   0.3 ]
 [1.   0.35]
 [1.   0.75]
 [1.   1.2 ]
 [1.   1.25]
 [1.   1.3 ]
 [1.   1.55]
 [1.   1.9 ]
 [1.   2.5 ]]


### QR Factorization

In [23]:
Q, R = np.linalg.qr(A)

In [24]:
print(Q)
print(R)

[[-0.31622777 -0.45357534]
 [-0.31622777 -0.35744184]
 [-0.31622777 -0.33559331]
 [-0.31622777 -0.16080513]
 [-0.31622777  0.03583158]
 [-0.31622777  0.0576801 ]
 [-0.31622777  0.07952862]
 [-0.31622777  0.18877124]
 [-0.31622777  0.3417109 ]
 [-0.31622777  0.60389318]]
[[-3.16227766 -3.53542642]
 [ 0.          2.28848421]]


### Solve square means problems to obtain $a_{1}$, $b_{1}$, $a_{2}$, $b_{2}$.

In [25]:
a1, b1 = np.linalg.solve(R, Q.T @ x)
a2, b2 = np.linalg.solve(R, Q.T @ y)

In [26]:
print((a1, b1))
print((a2, b2))

(0.15050752698027214, 1.8242329812341038)
(0.21519105774885652, 1.2386484277738314)


### Create functions with values obtained

In [27]:
f = lambda t: a1 + b1 * t
g = lambda t: a2 + b2 * t

### Create function of the differenciate to obtain t_tilde

In [28]:
def dife(a1, b1, a2, b2, p):
    return (p[0]*b1 - a1*b1 + p[1]*b2 - a2*b2)/(b1**2 + b2**2)

In [29]:
t_tilde = dife(a1,b1,a2,b2,p)
p2 = np.array([f(t_tilde), g(t_tilde)])
distance = np.linalg.norm(p-p2, ord=2, axis= 0)

In [30]:
print(f"t_tilde: {t_tilde}")
print(f"distance to line: {distance}")

t_tilde: 1.5909708197834906
distance to line: 0.984096962956381
